# Register plate to database

In [1]:
import sys
sys.path.append('/home/apaulson/repos/data-proc/')
import mysql_cnx as mc
%config Completer.use_jedi=False

import pandas as pd
import numpy as np
from sqlalchemy import text

In [2]:
engine.dispose()

NameError: name 'engine' is not defined

In [3]:
# engine=mc.create_mysql_cnx(db='QB3_HiTS_nightly')

engine=mc.create_mysql_cnx(db='QB3_HiTS')
with engine.connect() as conn:
    result=conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


In [4]:
# MetaData stores a collection of tables we want to work with
from sqlalchemy import MetaData
metadata = MetaData()

In [5]:
# reflection gets an existing table from the db
from sqlalchemy import Table

inv_plate=Table("INV_PLATE", metadata, autoload_with=engine)
inv_well=Table("INV_WELL", metadata, autoload_with=engine)
inv_comp_lot=Table("INV_COMP_LOT", metadata, autoload_with=engine)

In [6]:
inv_plate_cols=[x.name for x in inv_plate.columns]
inv_plate_cols=[x for x in inv_plate_cols if x != inv_plate.primary_key.columns[0].name]

inv_well_cols=[x.name for x in inv_well.columns]
inv_well_cols=[x for x in inv_well_cols if x != inv_well.primary_key.columns[0].name]

In [7]:
# insert
from sqlalchemy import insert
from datetime import datetime

In [8]:
datetime.now().strftime('%Y-%m-%d %H:%M:%S')

'2023-10-05 16:02:09'

In [9]:
inv_plate.primary_key.columns[0].name

'IP_ID'

## edit plate info here

In [10]:
platefile="/Users/apaulson/Downloads/TARG1 Hits2 Plate Map.xlsx"
platefile=platefile.replace("\\","/").replace('Z:','/mnt/z')
print(platefile)

/Users/apaulson/Downloads/TARG1 Hits2 Plate Map.xlsx


In [11]:
wells=pd.read_excel(platefile, index_col=0)
wells=wells.melt(id_vars=None, ignore_index=False)
wells=wells.reset_index()
wells.columns=['row','col','smdc']
wells

,row,col,smdc
0,A,1,701977.0
1,B,1,704905.0
2,C,1,708324.0
3,D,1,714200.0
4,E,1,718787.0
...,...,...,...
379,L,24,NaN
380,M,24,NaN
381,N,24,NaN
382,O,24,NaN


In [12]:
# wells=wells.sort_values('Plate_ID')

In [13]:
# barcodes=['EnamCov'+str(x).zfill(2)+'-MA01' for x in np.arange(1,24)]
# barcode_dict=dict(zip(wells.Plate_ID.unique(), barcodes))
barcode_dict={np.nan:'TARG1 2nd Hits'}

In [14]:
platelist=[]
for plid, barcode in barcode_dict.items():
    platedict={
     'IP_SOURCE_ID':None,
     'IP_GRID_NUM':None,
     'IP_IPT_ID': 8, # master plate
     'IP_IPF_ID':2, # 384w
     'IP_DESC':None,
     'IP_BARCODE':None,
     'IP_STATUS_ID':1,
     'IP_IV_ID':22, 
     'IP_VEN_PLID':None,
     'IP_ICO_ID':243, 
     'IP_COLLECTION':'Targ1 Cherry Pick', #name; not keyed so can be different but shouldn't be
     'IP_NOTES': None, #optional
     'IP_INS_BY': 569,
     'IP_INS_DATE':datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
     'IP_SCREENING_CENTER':'SMDC'
    }
    platedict['IP_GRID_NUM']=barcode.replace('-MA01','')
    platedict['IP_BARCODE']=barcode
    platedict['IP_VEN_PLID']=plid
    platelist.append(platedict)

In [15]:
plates=pd.DataFrame(platelist)
plates

,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER
0,None,TARG1 2nd Hits,8,2,None,TARG1 2nd Hits,1,22,NaN,243,Targ1 Cherry Pick,None,569,2023-10-05 16:03:37,SMDC


In [16]:
# check for important plate cols
assert set(plates.columns)==set(inv_plate_cols), "Missing info for INV_PLATE table"
assert len(plates[plates.IP_GRID_NUM.isna()])==0, "Missing GridNum"
assert len(plates[plates.IP_BARCODE.isna()])==0, "Missing Barcode"
assert len(plates[plates.IP_GRID_NUM.duplicated()])==0, "Duplicate gridnums"
assert len(plates[plates.IP_BARCODE.duplicated()])==0, "Duplicate barcodes"

In [17]:
# check existing IP_BARCODES
from sqlalchemy import select

stmt=select(inv_plate).where(inv_plate.c.IP_BARCODE.in_(tuple(plates.IP_BARCODE.tolist())))
with engine.connect() as conn:
    result=conn.execute(stmt)
    exists=result.rowcount>0

if exists:
    for i, row in enumerate(result):
        print(row)
        ip_id=row[0]
        assert i<0, "Warning: this barcode already exists"
else:
    # stmt=insert(inv_plate).values(plate)    
    # compiled=stmt.compile()
    # with engine.connect() as conn:
    #     result=conn.execute(stmt)
    #     conn.commit()
    # ip_id=result.inserted_primary_key
    ip_id=np.nan
    print('OK')

OK


In [18]:
import sqlalchemy
if isinstance(ip_id,sqlalchemy.engine.row.Row):
    ip_id=ip_id[0]
ip_id

nan

In [19]:
# # insert.. careful!!
# plates.to_sql(name='INV_PLATE', con=engine, if_exists = 'append', index=False)

1

In [20]:
# get IP_IDs
stmt=select(inv_plate).where(inv_plate.c.IP_BARCODE.in_(tuple(plates.IP_BARCODE.tolist())))
ip_ids=pd.read_sql(stmt, con=engine)
ip_ids

,IP_ID,IP_SOURCE_ID,IP_GRID_NUM,IP_IPT_ID,IP_IPF_ID,IP_DESC,IP_BARCODE,IP_STATUS_ID,IP_IV_ID,IP_VEN_PLID,IP_ICO_ID,IP_COLLECTION,IP_NOTES,IP_INS_BY,IP_INS_DATE,IP_SCREENING_CENTER
0,27861,None,TARG1 2nd Hits,8,2,None,TARG1 2nd Hits,1,22,None,243,Targ1 Cherry Pick,None,569,2023-10-05 16:03:37,SMDC


## edit well info here via plate map file

In [21]:
import plate_maps as pm

In [22]:
# wells
inv_well_cols

['IW_SOURCE_ID',
 'IW_IP_ID',
 'IW_BARCODE',
 'IW_WELL_INDEX',
 'IW_COORDINATES',
 'IW_ICL_ID',
 'IW_VOL_UL',
 'IW_CONC_UM',
 'IW_FINAL_CONC_NM',
 'IW_DILUTION_FACTOR',
 'IW_FREEZETHAW',
 'IW_INS_BY',
 'IW_INS_DATE',
 'IW_CONTAM',
 'IW_NOTES']

In [391]:
pf=pd.read_excel("/Users/apaulson/Downloads/Targ1_CP_dm.xlsx", sheet_name=None)

In [392]:
for i, row in pf['Info'].iterrows():
    wells=pm.make_dr_platemaps(pf, row, [("Amanda","Paulson"), ("Zain","Alam")], None)

TARG1HitCB


In [393]:
# wells=pd.read_excel(platefile)
wells['Plate_ID']=None
wells['IW_IP_ID']=wells.Plate_ID.map(dict(zip(ip_ids.IP_VEN_PLID, ip_ids.IP_ID)))
wells['IW_BARCODE']=wells.Plate_ID.map(dict(zip(ip_ids.IP_VEN_PLID, ip_ids.IP_BARCODE)))

# 40µL at 20mM in DMSO
wells['IW_VOL_UL']=5
wells['IW_CONC_UM']=10000 # this is storage conc not final conc in assay plate

wells.columns

Index(['SMDC_ID', 'Lot', 'Source plate', 'Source well', 'Destination plate',
       'Destination well', '[compound] uM', 'Assay ID', 'Plate_ID', 'IW_IP_ID',
       'IW_BARCODE', 'IW_VOL_UL', 'IW_CONC_UM'],
      dtype='object')

In [394]:
wells.columns=['ICL_IC_ID', 'ICL_LOT_NUM', 'Source_plate', 'Source_well', 'IP_BARCODE',
       'IW_COORDINATES', '[compound] uM', 'Assay ID', 'Plate_ID', 'IW_IP_ID',
       'IW_BARCODE', 'IW_VOL_UL', 'IW_CONC_UM']
wells=wells.replace('NA', np.nan)

In [396]:
# plate_map.columns=['Row','Col','ICL_IC_ID']
# plate_map['IW_COORDINATES']=plate_map.Row+plate_map.Col.astype(int).astype(str).str.zfill(2)
# plate_map['IW_IP_ID']=ip_id
# plate_map[['ICL_IC_ID','ICL_LOT_NUM']]=plate_map.ICL_IC_ID.str.split('-', expand=True)

# select lots by ic_id, lot tuple
tuplelist=wells[~wells.ICL_IC_ID.isna()][['ICL_IC_ID','ICL_LOT_NUM']].to_records(index=False)
ids=[]
with engine.connect() as conn:
    for x,y in tuplelist:
        stmt=select(inv_comp_lot.c.ICL_IC_ID, inv_comp_lot.c.ICL_LOT_NUM, inv_comp_lot.c.ICL_ID).where(inv_comp_lot.c.ICL_IC_ID==x, inv_comp_lot.c.ICL_LOT_NUM==y)
        for row in conn.execute(stmt):
           ids.append(row)

assert len(wells[~wells.ICL_IC_ID.isna()])==len(ids), "Num lot ID's does not match num SMDC ID's"

ids=pd.DataFrame(ids, columns=['ICL_IC_ID', 'ICL_LOT_NUM','ICL_ID'])
ids=ids.astype(int)
wells[['ICL_IC_ID','ICL_LOT_NUM']]=wells[['ICL_IC_ID','ICL_LOT_NUM']].astype(float)
wells=wells.merge(ids, how='left')
wells=wells.rename(columns={'ICL_ID':'IW_ICL_ID'})

In [397]:
len(wells.IW_COORDINATES.unique())

384

In [398]:
import string
rows=list(string.ascii_uppercase)[0:16]
cols=list(np.arange(1,25))

In [399]:
%time
allwells=[]
for row in rows:
    for col in cols:
        allwells.append(row+str(col).zfill(2))
len(allwells)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs


384

In [400]:
wellidx=dict(zip(allwells, np.arange(1,385)))

In [401]:
missingwells=list(set(allwells)-set(wells.IW_COORDINATES))
len(missingwells)

0

In [402]:
# # add missing wells back to each plate
# print(wells.shape)
# for plate, barcode in dict(zip(wells.IW_IP_ID,wells.IW_BARCODE)).items():
#     df=pd.DataFrame({'IW_IP_ID':plate, 'IW_BARCODE':barcode, 'IW_COORDINATES':missingwells})
#     wells=pd.concat([wells, df])
#     print(wells.shape)

In [403]:
# get source wells
tuplelist=wells[~wells.ICL_IC_ID.isna()][['Source_plate','Source_well']].to_records(index=False)
ids=[]
plates=[]
with engine.connect() as conn:
    for x,y in tuplelist:
        stmt=select(inv_plate.c.IP_ID).where(inv_plate.c.IP_BARCODE==x)
        ip_ids=[]
        for row in conn.execute(stmt):
            ip_ids.append(row)
        stmt=select(inv_well.c.IW_IP_ID, inv_well.c.IW_COORDINATES, inv_well.c.IW_ID).where(inv_well.c.IW_IP_ID==ip_ids[0][0], inv_well.c.IW_COORDINATES==y)
        for row in conn.execute(stmt):
           ids.append(row)
        plates.append((ip_ids[0][0], x))
assert len(wells[~wells.ICL_IC_ID.isna()])==len(ids)

ids=pd.DataFrame(ids, columns=['Source_IW_IP_ID', 'Source_well','IW_SOURCE_ID'])

# ids=ids.astype(int)
ids['Source_plate']=ids.Source_IW_IP_ID.map(dict(plates))

wells=wells.merge(ids, how="left")
wells[~wells.ICL_IC_ID.isna()]

,ICL_IC_ID,ICL_LOT_NUM,Source_plate,Source_well,IP_BARCODE,IW_COORDINATES,[compound] uM,Assay ID,Plate_ID,IW_IP_ID,IW_BARCODE,IW_VOL_UL,IW_CONC_UM,IW_ICL_ID,Source_IW_IP_ID,IW_SOURCE_ID
32,706120.0,1.0,04916CB-Eco1,P21,TARG1HitCB,A03,NaN,2439,None,27852,TARG1HitCB,5,10000,674482.0,27534.0,8485433.0
33,707527.0,1.0,04921CB-Eco1,L05,TARG1HitCB,B03,NaN,2439,None,27852,TARG1HitCB,5,10000,676054.0,27539.0,8487876.0
34,708792.0,1.0,04925CB-Eco1,H17,TARG1HitCB,C03,NaN,2439,None,27852,TARG1HitCB,5,10000,677320.0,27543.0,8489836.0
35,709573.0,1.0,04928CB-Eco1,C19,TARG1HitCB,D03,NaN,2439,None,27852,TARG1HitCB,5,10000,678101.0,27546.0,8491251.0
36,710556.0,1.0,04931CB-Eco1,I05,TARG1HitCB,E03,NaN,2439,None,27852,TARG1HitCB,5,10000,679084.0,27549.0,8492914.0
48,711055.0,1.0,04932CB-Eco1,N03,TARG1HitCB,A04,NaN,2439,None,27852,TARG1HitCB,5,10000,679583.0,27550.0,8493543.0
49,711218.0,1.0,04933CB-Eco1,M09,TARG1HitCB,B04,NaN,2439,None,27852,TARG1HitCB,5,10000,679746.0,27551.0,8494036.0
50,711258.0,1.0,04933CB-Eco1,E10,TARG1HitCB,C04,NaN,2439,None,27852,TARG1HitCB,5,10000,679786.0,27551.0,8493845.0
51,713617.0,1.0,04940CB-Eco1,N07,TARG1HitCB,D04,NaN,2439,None,27852,TARG1HitCB,5,10000,682145.0,27558.0,8497635.0
52,713945.0,1.0,04941CB-Eco1,P05,TARG1HitCB,E04,NaN,2439,None,27852,TARG1HitCB,5,10000,682474.0,27559.0,8498192.0


In [404]:
inv_well_cols

['IW_SOURCE_ID',
 'IW_IP_ID',
 'IW_BARCODE',
 'IW_WELL_INDEX',
 'IW_COORDINATES',
 'IW_ICL_ID',
 'IW_VOL_UL',
 'IW_CONC_UM',
 'IW_FINAL_CONC_NM',
 'IW_DILUTION_FACTOR',
 'IW_FREEZETHAW',
 'IW_INS_BY',
 'IW_INS_DATE',
 'IW_CONTAM',
 'IW_NOTES']

In [405]:
plate_map=wells[[x for x in wells.columns if x.startswith('IW_')]].copy()

plate_map['IW_WELL_INDEX']=plate_map.IW_COORDINATES.map(wellidx)

plate_map['IW_INS_BY']=569
plate_map['IW_INS_DATE']=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
plate_map['IW_CONTAM']=0

plate_map=plate_map.sort_values(['IW_IP_ID', 'IW_COORDINATES'])
plate_map.head()

,IW_COORDINATES,IW_IP_ID,IW_BARCODE,IW_VOL_UL,IW_CONC_UM,IW_ICL_ID,IW_SOURCE_ID,IW_WELL_INDEX,IW_INS_BY,IW_INS_DATE,IW_CONTAM
0,A01,27852,TARG1HitCB,5,10000,NaN,NaN,1,569,2023-09-26 18:04:14,0
16,A02,27852,TARG1HitCB,5,10000,NaN,NaN,2,569,2023-09-26 18:04:14,0
32,A03,27852,TARG1HitCB,5,10000,674482.0,8485433.0,3,569,2023-09-26 18:04:14,0
48,A04,27852,TARG1HitCB,5,10000,679583.0,8493543.0,4,569,2023-09-26 18:04:14,0
64,A05,27852,TARG1HitCB,5,10000,683645.0,8500071.0,5,569,2023-09-26 18:04:14,0


In [406]:
plate_map.shape

(384, 11)

In [407]:
# check IP_ID doesn't exist in INV_WELL yet
stmt=select(inv_well).where(inv_well.c.IW_IP_ID.in_(tuple(wells.IW_IP_ID.unique())))
with engine.connect() as conn:
    result=conn.execute(stmt)
    assert result.rowcount==0, "Wells exist for these plates"

In [408]:
plate_map.shape

(384, 11)

In [409]:
# # insert... careful here
# plate_map.to_sql(name='INV_WELL', con=engine, if_exists = 'append', index=False)

384

In [410]:
engine.dispose()

In [499]:
# plate_map.to_csv('EnamCov_wells.csv')